# A (Detailed) Peek into PCA and SVD

2016/10/05 by Bowen Li; updated: 2020/02/08

## Outline
- [What is Principal Component Analysis (PCA)?](#What-is-PCA?)
  * [Assumptions for PCA](#Assumptions-for-PCA)
  * [PCA Computation](#PCA-Computation)
  * [Eigenvector-Decomposition](#Eigenvector-Decomposition)
  * [Summary of PCA](#Summary-of-PCA)
- [What is Singular Value Decomposition (SVD)?](#What-is-SVD?)
  * [Matrix Version of SVD](#Matrix-Version-of-SVD)
- [PCA and SVD](#PCA-and-SVD)
  * [How Many Pricipal Components We Shall Use?](#How-Many-Pricipal-Components-We-Shall-Use?)
  * [Reduced Rank Approximation by SVD](#Reduced-Rank-Approximation-by-SVD)
- [References](#References)

## What is PCA?

The motivation of PCA is to identify the most meaningful basis to re-express data which can maximize the [signal-to-noise ratio (SNR):](https://en.wikipedia.org/wiki/Signal-to-noise_ratio) $\sigma^2_{signal} / \sigma^2_{noise}$.

<img src="./images/pca_snr.png" alt="Drawing" style="width: 300px;"/>

- Maximize the signal, measured by the magnitude of variance; large variance corresponds to interesting structure.
- Minimize redundancy (i.e. noise), measured by the magnitude of covariance.

Let the observed data be

$$
\underset{(n \times p)}X = \{x_{ij}\} =
\begin{bmatrix}
x_{11} & \cdots & x_{1p} \\
\vdots &        & \vdots \\ 
x_{n1} & \cdots & x_{np}
\end{bmatrix}
= [x_1, \cdots, x_p]
$$

where each row represents an example measured on $p$-dimensional features, and each column, $x_j$, represents normalized feature with zero means, $E(x_j) = 0$, and variance one, $Var(x_j) = 1$.

$$
x_j = 
\begin{bmatrix}
x_{1j} \\
\vdots \\ 
x_{nj}
\end{bmatrix}
$$

The covariance matrix of $X$ is

$$
\underset{(p \times p)}{\Sigma_X} = \frac{1}{n}X^{T}X 
= \frac{1}{n}
\begin{bmatrix}
x^T_{1} \\
\vdots \\ 
x^T_{p}
\end{bmatrix}
[x_1,...,x_p]
= \frac{1}{n}
\begin{bmatrix}
x^T_{1}x_1 & \cdots & x^T_{1}x_p \\
\vdots     &        & \vdots \\ 
x^T_{p}x_1 & \cdots & x^T_{p}x_p
\end{bmatrix}
$$

Our goal is to find some transformation, $Y = f(X)$, of X such that covariance of $Y$ is a *diagonal* matrix,

$$
\underset{(p \times p)}{\Sigma_Y} = \frac{1}{n}Y^{T}Y
$$

There are many transformations for diagonalizing the convariance matrix $\Sigma_X$, PCA arguably selects the easiest one.

### Assumptions for PCA
- Linear transformation: Let $\underset{p \times p}E = [e_1,...,e_p]$,

$$
Y = XE
$$

- Large variance have important structure.
- The principal components are orthogonal. 

The last assumption provides an intuitive simplification that makes PCA soluble with linear algebra decomposition techniques. 

### PCA Computation

Based on the above assumptions, define
- The 1st principal component = linear combination $Xe_1$ that maximizes $Var(Xe_1)$, subject to $e^T_1e_1 = 1$.
- The 2nd principal component = linear combination $Xe_2$ that maximizes $Var(Xe_2)$, subject to $e^T_2e_2 = 1$ and $Cov(Xe_1, Xe_2) = 0$.
- The $j$th principal component = linear combination $Xe_j$ that maximizes $Var(Xe_j)$, subject to $e^T_j e_j = 1$ and $Cov(Xe_k, Xe_j) = 0$ for $k < j$.

### Eigenvector Decomposition

The [eigenvector decomposition](https://en.wikipedia.org/wiki/Eigendecomposition_of_a_matrix) is also called the [spectral secomposition](https://en.wikipedia.org/wiki/Spectral_theorem). Suppose $Y = XE$, its covariance matrix

$$
\Sigma_Y = \frac{1}{n}Y^TY = \frac{1}{n}(XE)^T(XE) = E^T \left(\frac{1}{n}X^TX \right)E
$$

Since $\frac{1}{n}X^TX$ is a *symmetric* matrix, it can be diagonalized by a matrix of orthonormal eigenvectors using the spectral decomposistion,

$$
\frac{1}{n}X^TX = VDV^T
$$

where $D = diag(d_1,...,d_p)$ is a diagonal matrix with rank-ordered set of eigenvalues, $d_1 \ge d_2 \ge ... \ge d_p$, and $V = [v_1,...,v_p]$ is the matrix of the corresponding eigenvectors with $V^TV=VV^T = I$.

*Sketch of proof:* From the spectral decomposition, for all $j$, we have 

$$
\frac{1}{n}X^T X v_j = d_j v_j
$$

Then we have $\frac{1}{n}X^T X V = V D$ which implies $\frac{1}{n}X^T X = V D V^T$.

*Alternative proof:* We can interpret PCA as finding a unit vector $u$, with $|u| = 1$, which maximizes variance of the projection of all example $x_i$'s onto $u$,

$$
\frac{1}{n} \sum_{i=1}^n (x_i^T u)^2 
= \frac{1}{n} \sum_{i=1}^n (x_i^T u)^T (x_i^T u)
= u^T \big(\frac{1}{n} \sum_{i=1}^n x_i x_i^T \big) u
$$

But let's digress a little to ask why the variance of the projection of all examples $x_i$'s onto $u$ has form like the above? First, suppose the angle between $x_i$ and $u$ is $\theta$, then the projection of each example $x_i$ onto $u$ is

$$
x_i' = x_i cos\theta
= x_i \frac{x_i^T u}{|x_i||u|}
= x_i \frac{x_i^T u}{|x_i|}
$$

since $u$ is a unit vector. Then the length of the projection of one example $x_i$'s onto $u$ is

$$
\left[ x_i'^T x_i' \right]^{1/2}
= \left[ \left( x_i \frac{x_i^T u}{|x_i|} \right)^T \left( x_i \frac{x_i^T u}{|x_i|} \right) \right]^{1/2}
= \left[ \left( \frac{x_i^T u}{|x_i|} \right) x_i^T x_i \left( \frac{x_i^T u}{|x_i|} \right) \right]^{1/2}
= \left[ \left(x_i^T u \right)^2 \right]^{1/2}
= x_i^T u
$$

since $x_i^T x_i = |x_i|^2$. Finally, the projection of one example $x_i$'s onto $u$ is the distance $x_i^T u$ from the origin.

Now let's go back to our optimization problem, we would like to solve

$$
max_u u^T \big(\frac{1}{n} \sum_{i=1}^n x_i x_i^T \big) u
$$

such that $u^T u = 1$. Using the [lagrange multiplier method](https://en.wikipedia.org/wiki/Lagrange_multiplier), the objective becomes

$$
max_u L(u, \lambda) = u^T \big(\frac{1}{n} \sum_{i=1}^n x_i x_i^T \big) u - d (u^T u - 1)
$$

Take partial derivative with respect to $u$ we have

$$
\frac{\partial}{\partial u} L(u, \lambda) = \frac{1}{n} X^T X u - d u := 0
$$

Hence, we are solving $\frac{1}{n} X^T X u = d u$, and the eigenvector decomposition result follows.

From the above results we can select $E = V$. Then since $V^TV=VV^T = I$, $\Sigma_Y$ now is a diagonal matrix

$$
\Sigma_Y = E^T \left( \frac{1}{n}X^T X \right) E
    = V^T \left( V D V^T \right) V = D
$$

Note that the eigenvalue $d_j$ is the variance of transformed feature $y_j$.

### Summary of PCA
- Substract off the mean of each feature: $x_j \leftarrow x_j - \overline x_j$, and divide the standard deviation: $x_j \leftarrow x_j/\sigma_j$
- Compute the eigenvectors of covariance matrix, $\Sigma_X = \frac{1}{n}X^TX$
- The ith principal components of $X$ is the covariance matrix $\Sigma_X$'s eigenvector $v_i$
- The ith diagonal value of the transformed covariance matrix $\Sigma_Y$ is the variance of $X$ along $v_i$ 

## What is SVD?
First recall the eigenvector decomposition: $\frac{1}{n}X^T X v_j = d_j v_j$. Define
- Singular values: square root of eigenvalue (variance) $\lambda_j = \sqrt{d_j}$, for $j = 1,...,k$.
- Matrix $U = [ u_1,...,u_2 ]$, with $u_j$ defined by

$$
u_j = \frac{1}{\lambda_j}\left( \frac{1}{\sqrt{n}}X \right) v_j
$$

where $v_j$ is the eigenvector.

Then we have
- (1) $U$ is *orthonormal* matrix:
$u_i^T u_j = 
\begin{cases} 
1, \text{if } i = j \\
0, \text{otherwise}
\end{cases}
$
- (2) $\lVert X v_i\rVert = \lambda_i$

*Sketch of proof:* $u_i^T u_j = \frac{1}{n} \left( \frac{1}{\lambda_i} X v_i \right)^T \left( \frac{1}{\lambda_j} X v_j \right) = \frac{1}{\lambda_i \lambda_j} v_i^T \frac{1}{n} X^T X v_j = \frac{1}{\lambda_i \lambda_j} v_i^T d_j v_j = \frac{\lambda_j}{\lambda_i} v_i^T v_j$. The first result follows. Further, we can obtain the second result similarly.

By rewriting we have

$$
\left( \frac{1}{\sqrt{n}}X \right) v_j = \lambda_j u_j
$$

That is, normalized $X$ multiplied by an eigenvector $v_j$ of covariance of $X$, $\frac{1}{n} X^T X$, is equal to a scalar $\lambda_j$ times another vector, $u_j$.

### Matrix Version of SVD

- By constructing a new *diagonal* matrix of singular values, $\Sigma = diag(\lambda_1,...,\lambda_p)$, where $\lambda_1 \ge \lambda_2 \ge ... \ge \lambda_p$.
- Define $V = [v_1,...,v_p]$ and $U = [u_1,...,u_p]$.

$$
\left( \frac{1}{\sqrt{n}}X \right) V = U \Sigma,
\frac{1}{\sqrt{n}}X = U \Sigma V^T
$$


Hence, *any* arbitrary matrix $\frac{1}{\sqrt{n}}X$ can be transformed to
- an orthogonal matrix (*rotation*): $U$
- a diagonal matrix (*stretch*): $\Sigma$
- another orthogonal matrix (*second rotation*): $V$

## PCA and SVD
From SVD, we have

$$
\frac{1}{n} X^T X = \left( \frac{1}{\sqrt{n}}X \right)^T \left( \frac{1}{\sqrt{n}}X \right)
                  = \left( U \Sigma V^T \right)^T \left( U \Sigma V^T \right) \\
                  = \left( V \Sigma U^T U \Sigma V^T \right)
                  = V \Sigma^2 V^T \equiv V D V^T
$$

That is, squared singular value is equal to variance of $X$ along eigenvector $v_j$, $\lambda_j^2 = d_j$.

### How Many Pricipal Components We Shall Use?
- Total variances: $\sum_{j=1}^p \lambda_j^2$.
- *Scree plot:* Choose the number of principal component by *elbow method.*

<img src="./images/pca_scree_plot.png" alt="Drawing" style="width: 200px;"/>

### Reduced Rank Approximation by SVD
Recall from SVD, 

$$
\frac{1}{\sqrt{n}}X = U \Sigma V^T = \sum_{j=1}^p \lambda_j u_j v_j^T
$$

Let $s < p = rank(X)$. Then the reduced rank-$s$ least squares approximation to $\frac{1}{\sqrt{n}} X$ is

$$
\frac{1}{\sqrt{n}}\widehat{X} = \sum_{j=1}^s \lambda_j u_j v_j^T
$$

which minimizes

$$
\frac{1}{n} \sum_{i=1}^n \sum_{j=1}^p \left( x_{ij} - \widehat{x}_{ij} \right)^2
= tr \left[ \left(\frac{1}{\sqrt{n}} (X - \widehat{X}) \right) \left(\frac{1}{\sqrt{n}} (X - \widehat{X}) \right)^T \right]
$$

over all matrices $\widehat{X}$ having rank no greater than $s$.

---

Cited as:

```
@article{li2016pcasvd,
  title   = "A (Detailed) Peek into PCA and SVD",
  author  = "Li, Bowen",
  journal = "https://bowen0701.github.io/",
  year    = "2016",
  url     = "https://bowen0701.github.io/2016/10/05/pca-svd/"
}
```

*If you notice mistakes and errors in this post, don’t hesitate to contact me at [bowen0701 at gmail dot com] and I would be grateful to be able to correct them.*

See you in the next post. :-)

## References

[1] Shlens (arXiv, 2014). A Tutorial on Principal Component Analysis.

[2] Johnson & Wichern (2002). Applied Multivariate Statistical Analysis.## References